### Nonlinear Function

Given a $\phi$ and $\phi^{-1}$ (trained in Quantum autoencoder) such that $\phi^{-1}\circ\phi|\alpha\rangle = |\alpha\rangle$ and $dim(\phi|\alpha\rangle) < dim(|\alpha\rangle)$, we want to find a function $f$ such that $\phi^{-1}\circ f\circ\phi|\alpha_k\rangle = |\alpha_{k+1}\rangle$.

In [1]:
from contextlib import redirect_stdout  #Used for writing model architecture to datafiles
import matplotlib.pyplot as plt         
from datetime import date               #Used for datafiles
import tensorflow as tf
from time import sleep
import numpy as np
import os


#Some GPU configuration
#Always uses the 1st GPU avalible (if avalible) unless 1st line is uncommented, in which case no GPU is used

#tf.config.set_visible_devices([], 'GPU') #uncomment to set tensorflow to use CPU
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) != 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [4]:
Phi = tf.keras.models.load_model('../Autoencoder/Autoencoder_Trials/models/trial12e300Phi.h5')
Phi_inv = tf.keras.models.load_model('../Autoencoder/Autoencoder_Trials/models/trial12e300Phi_inv.h5')

In [5]:
def ideal_phi(states):
    theta = np.math.atan2(states[1],states[0]) - np.math.atan2(states[3],states[2])
    r = np.sqrt(states[0]*states[0]+states[1]*states[1])
    return np.array([r, theta])

def ideal_phi_inv(compressed):
    r, theta = compressed[0], compressed[1]
    
    alpha = r*np.cos(theta)
    beta = r*np.sin(theta)
    gamma = np.sqrt(1-r*r)
    delta = 0.
    return np.array([alpha, beta, gamma, delta])

def get_relative_phase(vector):
    '''Returns the relative phase between
    the two complex components of a two
    complex dimensional vector
    Assumes the vector is passed in as a 
    four dimensional real row vector of form
    [real1, imag1, real2, imag2]
    '''
    

    #Tensorflow likes to return a list of a single
    #element sometimes, which breaks this function
    #This does not happen during training, only when
    #manually run on a single vector
    if vector.shape == (4,):
        return tf.atan2(vector[1], vector[0])%np.pi - tf.atan2(vector[3], vector[2])%np.pi

    return tf.atan2(vector[:,1],vector[:,0])%np.pi - tf.atan2(vector[:,3],vector[:,2])%np.pi

In [10]:
inputs = tf.keras.Input(shape=Phi.layers[-1].output_shape[-1])

nonlinear_layer_1 = tf.keras.layers.Dense(8, activation='selu', name='nonlinear_layer_1')(inputs)
nonlinear_layer_2 = tf.keras.layers.Dense(16, activation='selu', name='nonlinear_layer_2')(nonlinear_layer_1)
nonlinear_layer_3 = tf.keras.layers.Dense(32, activation='selu', name='nonlinear_layer_3')(nonlinear_layer_2)
nonlinear_layer_4 = tf.keras.layers.Dense(16, activation='selu', name='nonlinear_layer_4')(nonlinear_layer_3)
nonlinear_layer_5 = tf.keras.layers.Dense(8, activation='selu', name='nonlinear_layer_5')(nonlinear_layer_4)
evolved = tf.keras.layers.Dense(Phi.layers[-1].output_shape[-1], activation='selu', name='evolved_state_layer')(nonlinear_layer_5)

NonlinearEvolution = tf.keras.Model(inputs=inputs, outputs=evolved)

In [26]:
x1, y1, x2, y2 = np.random.uniform(low=-1, high=1, size=4)
norm = np.sqrt(x1*x1+x2*x2+y1*y1+y2*y2)
state = 1/norm * np.array([x1,y1, x2,y2])
print(state)
print(ideal_phi(state))
print(ideal_phi_inv(ideal_phi(state)))
print(get_relative_phase(state) - get_relative_phase(ideal_phi_inv(ideal_phi(state))))

[ 0.44368029  0.01221893  0.76786864 -0.46192668]
[0.44384851 0.56910604]
[0.37389047 0.23918073 0.89610183 0.        ]
tf.Tensor(0.0, shape=(), dtype=float64)
